In [ ]:
from functools import partial

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler

from preprocessing import preprocess
from utils import read_train, build_feature_path, read_train_no_validation, read_test, read_validation
import autosklearn


import pickle

import autosklearn.classification

from sklearn.metrics import classification_report


In [2]:
print('read training data')
train = read_train()
features = [
'bert_avg_all_but_first_binary_scaled',
'boosters_selected',
'char_prediction',
'hashtags_selected',
'hedges_selected',
'mentions_total',
'female_words_selected',
'male_words_selected',
'most_similar_scale_selected',
'perspective_selected',
'perspective_difference_selected',
'senpai_selected',
'senpai_unclustered_selected',
'sif',
'vader_selected',
]
for feature in features:
    print('read precomputed feature', feature)
    feature_path = build_feature_path('TRAINING_REL', feature)
    feature_df = pd.read_csv(feature_path, index_col='id')
    feature_df.columns = [feature + "_" + column for column in feature_df.columns]
    train = pd.merge(train, feature_df, how='left', left_index=True, right_index=True)
print('encode language')
language_le = LabelEncoder()
train['language'] = language_le.fit_transform(train.language)
print('preprocess text')
train['text'] = train.text.apply(partial(preprocess, fix_encoding=True))
features += ['language', 'text']

train_ = train.loc[read_train_no_validation().index]
X = train_[[column for column in train.columns if any(column.startswith(feature) for feature in features)]]

y = train_.task2.values
y_le = LabelEncoder()
y = y_le.fit_transform(y)

print('X.shape', X.shape, 'y.shape', y.shape, 'unique y', np.unique(y))
labels = y_le.classes_


read training data
read precomputed feature bert_avg_all_but_first_binary_scaled
read precomputed feature boosters_selected
read precomputed feature char_prediction
read precomputed feature hashtags_selected
read precomputed feature hedges_selected
read precomputed feature mentions_total
read precomputed feature female_words_selected
read precomputed feature male_words_selected
read precomputed feature most_similar_scale_selected
read precomputed feature perspective_selected
read precomputed feature perspective_difference_selected
read precomputed feature senpai_selected
read precomputed feature senpai_unclustered_selected
read precomputed feature sif
read precomputed feature vader_selected
encode language
preprocess text
X.shape (5581, 2976) y.shape (5581,) unique y [0 1 2 3 4 5]


In [ ]:
char = ColumnTransformer(transformers=[('cv', Pipeline(steps=[('cv', CountVectorizer(analyzer='char',
                                                                  ngram_range=(3, 4))),
                                                                  ('fs',SelectFromModel(estimator=MultinomialNB())  # use multitask in case of task2
                                        )]), 'text')],
                             remainder='passthrough'
                             )
X_char = char.fit_transform(X, y)


In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=60*60*8,
#     per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_tmp5',
    output_folder='/tmp/autosklearn_out5',
    n_jobs=10,
    # Each one of the 4 jobs is allocated 6GB
    memory_limit=2*3072,
    seed=5,
    
#     ml_memory_limit=600*1024,
#     ensemble_size=1,
#     ensemble_memory_limit=7*1024,
#     initial_configurations_via_metalearning=0,
#     include_preprocessors=["no_preprocessing"],
    delete_output_folder_after_terminate=True,
    delete_tmp_folder_after_terminate=True,
)
automl.fit(X_char, y)


In [5]:
with open('task2.pickle','wb+') as f:
    pickle.dump(automl, f)


In [ ]:
validation = train.loc[read_validation().index]
X = validation[[column for column in train.columns if any(column.startswith(feature) for feature in features)]]

y = validation.task2.values
y = y_le.transform(y)

print('X.shape', X.shape, 'y.shape', y.shape, 'unique y', np.unique(y))
labels = y_le.classes_
X_char = char.transform(X)
y_pred = automl.predict(X_char)

In [7]:
print(classification_report(y_true=y, y_pred=y_pred, target_names=labels))


                              precision    recall  f1-score   support

      ideological-inequality       0.63      0.44      0.52       173
misogyny-non-sexual-violence       0.51      0.33      0.40       137
                  non-sexist       0.65      0.88      0.75       720
             objectification       0.59      0.26      0.36       100
             sexual-violence       0.56      0.50      0.53       104
      stereotyping-dominance       0.54      0.27      0.36       162

                    accuracy                           0.63      1396
                   macro avg       0.58      0.45      0.49      1396
                weighted avg       0.61      0.63      0.60      1396



In [15]:
y_le.inverse_transform([0, 1,2, 3, 4, 5])

array(['ideological-inequality', 'misogyny-non-sexual-violence',
       'non-sexist', 'objectification', 'sexual-violence',
       'stereotyping-dominance'], dtype=object)

In [17]:
print(classification_report(y_true=(y!=2), y_pred=(y_pred!=2), target_names=['non-sexist', 'sexist']))


              precision    recall  f1-score   support

  non-sexist       0.65      0.88      0.75       720
      sexist       0.80      0.50      0.62       676

    accuracy                           0.70      1396
   macro avg       0.73      0.69      0.68      1396
weighted avg       0.72      0.70      0.69      1396



In [ ]:
print('read test data')
test = read_test()
for feature in features:
    if feature in ['language', 'text']:continue
    print('read precomputed feature', feature)
    feature_path = build_feature_path('TEST_REL', feature)
    feature_df = pd.read_csv(feature_path, index_col='id')
    feature_df.columns = [feature + "_" + column for column in feature_df.columns]
    test = pd.merge(test, feature_df, how='left', left_index=True, right_index=True)
print('encode language')
test['language'] = language_le.fit_transform(test.language)
print('preprocess text')
test['text'] = test.text.apply(partial(preprocess, fix_encoding=True))

X_test = test[[column for column in train.columns if any(column.startswith(feature) for feature in features)]]


In [ ]:

X_test_char = char.transform(X_test)


In [10]:
y_test_pred = automl.predict(X_test_char)

In [11]:
y_test_pred= y_le.inverse_transform(y_test_pred)

In [12]:
np.unique(y_test_pred)

array(['ideological-inequality', 'misogyny-non-sexual-violence',
       'non-sexist', 'objectification', 'sexual-violence',
       'stereotyping-dominance'], dtype=object)

In [13]:
pd.DataFrame(pd.Series(y_test_pred, index=test.index, name='task2')).reset_index().transform({'id':lambda x: "{:06d}".format(x),
                                                                                             'task2':lambda x:x}).to_csv('task2.csv', index=False)

In [14]:
print(automl.show_models())

[(0.140000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'passive_aggressive', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'normalize', 'feature_preprocessor:__choice__': 'liblinear_svc_preprocessor', 'classifier:passive_aggressive:C': 0.015576263428050963, 'classifier:passive_aggressive:average': 'False', 'classifier:passive_aggressive:fit_intercept': 'True', 'classifier:passive_aggressive:loss': 'hinge', 'classifier:passive_aggressive:tol': 0.0014769179509443724, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.35734816689061744, 'feature_preprocessor:liblinear_svc_preprocessor:C': 15851.390191976288, 'feat

In [18]:
results_df = pd.DataFrame(pd.Series(y_test_pred, index=test.index, name='task2')).reset_index().transform({'id':lambda x: "{:06d}".format(x),
                                                                                             'task2':lambda x:x})
results_df['test_case'] = 'EXIST2021'
results_df[['test_case', 'id', 'task2']].to_csv('task2.tsv', sep='\t', header = None, index=False)